In [1]:
ad_genes = "APP,HFE,MPO,NOS3,PLAU,ABCA7,PSEN2,PSEN1,APOE"
schizo_genes = "APOL2,APOL4,CHI3L1,COMT,DAOA,DISC2,DRD3,HTR2A,MTHFR,RTN4R,SYN2,SHANK3,DISC1,RBM12,NRXN1,SLC1A1,PRODH,NRG1"

ad_dict = {
    # Alzheimer Disease
    "APP": "9606.ENSP00000284981",
    "HFE": "9606.ENSP00000417404",
    "MPO": "9606.ENSP00000225275",
    "NOS3": "9606.ENSP00000297494",
    "PLAU": "9606.ENSP00000361850",
}

ad_related_dict = {
    # AD Related
    "ABCA7": "9606.ENSP00000263094",
    "PSEN1": "9606.ENSP00000326366",
    "PSEN2": "9606.ENSP00000355747",
    "APOE": "9606.ENSP00000252486",
}

ad_full = ad_dict | ad_related_dict

schiz_dict = {
    # Schizophrenia
    "APOL2": "9606.ENSP00000249066",
    "APOL4": "9606.ENSP00000338260",
    "CHI3L1": "9606.ENSP00000255409",
    "COMT": "9606.ENSP00000354511",
    "DAOA": "9606.ENSP00000483757",
    "DISC1": "9606.ENSP00000355593",
    "DRD3": "9606.ENSP00000373169",
    "HTR2A": "9606.ENSP00000437737",
    "MTHFR": "9606.ENSP00000365777",
    "RTN4R": "9606.ENSP00000043402",
    "SYN2": "9606.ENSP00000480050",
}

schiz_related_dict = {
    # Schizophrenia Related
    "RBM12": "9606.ENSP00000363228",
    "NRXN1": "9606.ENSP00000385142",
    "SLC1A1": "9606.ENSP00000262352",
    "PRODH": "9606.ENSP00000481127",
    "NRG1": "9606.ENSP00000384620",
}

schiz_full = schiz_dict | schiz_related_dict

In [2]:
from graph_tool.all import *

g = load_graph("/data.graphml")

print(g)

<Graph object, undirected, with 19382 vertices and 5968680 edges, 1 internal vertex property, 4 internal edge properties, at 0x7fa09b5f1370>


In [ ]:
print(g.vertex_index)
for v in g.vertices():
    print(v)

In [3]:
# Centrality
vp_betweenness, ep_betweenness = graph_tool.centrality.betweenness(g)
print(f"vertex p betweenness = {vp_betweenness}")
print(f"edge p betweenness = {ep_betweenness}")

vp_closness = graph_tool.centrality.closeness(g)
print(f"vertex p closeness = {vp_closness}")

vertex p betweenness = <VertexPropertyMap object with value type 'double', for Graph 0x7fa09b5f1370, at 0x7fa067de2190>
edge p betweenness = <EdgePropertyMap object with value type 'double', for Graph 0x7fa09b5f1370, at 0x7fa067de2e50>
vertex p closeness = <VertexPropertyMap object with value type 'double', for Graph 0x7fa09b5f1370, at 0x7fa09b5cb910>


In [5]:
# Clustering

local_clustering = graph_tool.clustering.local_clustering(g)

print(graph_tool.stats.vertex_average(g, local_clustering))

global_clustering = graph_tool.clustering.global_clustering(g)

print(global_clustering)

(0.1952338728921458, 0.0006214113968411895)
(0.19299713533367516, 0.0010956810919999342)


In [6]:
# Correlations
assort = graph_tool.correlations.assortativity(g, "total")
print(assort)

(3.561537713627508e-05, 1.0387960155481485e-05)


In [8]:
schiz_res = {}
ad_res = {}

# print(g.list_properties())
prop = g.vertex_properties["_graphml_vertex_id"]

for key, value in schiz_dict.items():
    schiz_vertex = graph_tool.util.find_vertex(g, prop, key)[0]
    print(f"Schiz: found vertex for {key} - {schiz_vertex}")
    schiz_res[key] = schiz_vertex


for key, value in ad_dict.items():
    ad_vertex = graph_tool.util.find_vertex(g, prop, key)[0]
    print(f"AD: found vertex for {key} - {ad_vertex}")
    ad_res[key] = ad_vertex


Schiz: found vertex for APOL2 - 13385
Schiz: found vertex for APOL4 - 18099
Schiz: found vertex for CHI3L1 - 3394
Schiz: found vertex for COMT - 1486
Schiz: found vertex for DAOA - 15629
Schiz: found vertex for DISC1 - 12924
Schiz: found vertex for DRD3 - 6981
Schiz: found vertex for HTR2A - 8935
Schiz: found vertex for MTHFR - 5774
Schiz: found vertex for RTN4R - 4843
Schiz: found vertex for SYN2 - 8892
AD: found vertex for APP - 1501
AD: found vertex for HFE - 1856
AD: found vertex for MPO - 5548
AD: found vertex for NOS3 - 5768
AD: found vertex for PLAU - 10331


In [14]:
# Print Stats from our Vertices
combined = schiz_res | ad_res
for key, value in combined.items():
    print(f"betweenness of {key}: {vp_betweenness[value]}")
    print(f"closeness of {key}: {vp_closness[value]}")
    print(f"degree of {key}: {g.get_total_degrees([value])}")
    
    
print(f"edge degree average whole graph: {graph_tool.stats.vertex_average(g, 'total')}")

betweenness of APOL2: 3.752353289463026e-05
closeness of APOL2: 0.4878668881840608
degree of APOL2: [287]
betweenness of APOL4: 1.0188693396455386e-05
closeness of APOL4: 0.4719361044147369
degree of APOL4: [150]
betweenness of CHI3L1: 4.208424084318323e-05
closeness of CHI3L1: 0.5054111142983806
degree of CHI3L1: [745]
betweenness of COMT: 0.00016899743429551862
closeness of COMT: 0.5137033502968618
degree of COMT: [1228]
betweenness of DAOA: 5.048852718436345e-06
closeness of DAOA: 0.48015558418392623
degree of DAOA: [174]
betweenness of DISC1: 0.0001236534472975964
closeness of DISC1: 0.5114395038923341
degree of DISC1: [1047]
betweenness of DRD3: 2.7550912301138258e-05
closeness of DRD3: 0.5003872766704534
degree of DRD3: [549]
betweenness of HTR2A: 3.6927097011795e-05
closeness of HTR2A: 0.506918107394136
degree of HTR2A: [855]
betweenness of MTHFR: 0.00016848951117455745
closeness of MTHFR: 0.5170334800586902
degree of MTHFR: [1399]
betweenness of RTN4R: 5.934462972672222e-05
clo

In [ ]:
# Draw with Modularity
state = graph_tool.inference.minimize.minimize_blockmodel_dl(g)
print(state)
state.draw(pos=g.vp.pos, output="blockmodel.svg")

In [ ]:
# Draw Random
pos = graph_tool.draw.random_layout(g, dim=2)
graph_tool.draw.graph_draw(g, pos=pos, output="graph-draw-random.pdf")

# OR
# graph_tool.draw.graph_draw(g, vertex_text=g.vertex_index, output="test.pdf")

In [ ]:
# Draw with Fruchterman Reingold Layout
pos = graph_tool.draw.fruchterman_reingold_layout(g, n_iter=1000)
graph_tool.draw.graph_draw(g, pos=pos, output="graph-draw-fr.pdf")

In [ ]:
# Minimal Spanning Treen
from numpy.random import random

tree = graph_tool.topology.min_spanning_tree(g)
u = gt.GraphView(g, efilt=tree)
gt.graph_draw(u, pos=pos, output="triang_min_span_tree.pdf")